# Fine-Tuning Neural Network Hyperparameters

In [2]:
import warnings
warnings.filterwarnings(action="ignore", category=DeprecationWarning)

import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from IPython.display import display
import numpy as np, pandas as pd
import keras
import time

## Building a Regression MLP Using the Sequential API

In [3]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error as mse

In [4]:
housing = fetch_california_housing()
# print(housing)

In [5]:
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [8]:
X_train.shape[1:]

(8,)

In [15]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [16]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [11]:
keras_reg.fit(X_train, y_train, epochs=100,
                validation_data=(X_valid, y_valid),
                callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


C:\Users\reesh\anaconda3\envs\hands\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


363/363 [==============================] - 4s 5ms/step - loss: 1.4082 - val_loss: 0.7686
Epoch 2/100
363/363 [==============================] - 2s 5ms/step - loss: 0.6836 - val_loss: 0.6567
Epoch 3/100
363/363 [==============================] - 1s 4ms/step - loss: 0.6108 - val_loss: 0.5953
Epoch 4/100
363/363 [==============================] - 1s 4ms/step - loss: 0.5631 - val_loss: 0.5518
Epoch 5/100
363/363 [==============================] - 1s 4ms/step - loss: 0.5285 - val_loss: 0.5287
Epoch 6/100
363/363 [==============================] - 1s 4ms/step - loss: 0.5027 - val_loss: 0.5170
Epoch 7/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4861 - val_loss: 0.4889
Epoch 8/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4714 - val_loss: 0.4724
Epoch 9/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4622 - val_loss: 0.4647
Epoch 10/100
363/363 [==============================] - 1s 4ms/step - loss: 0.4536 - val_loss: 0.4638
E

NameError: name 'X_new' is not defined

In [13]:
X_new = X_test[:3] # pretend these are new instances
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

1/1 [==============================] - 0s 77ms/step


In [ ]:
y_test[:3]

In [ ]:
y_pred

In [ ]:
# reciprocal()

In [14]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/100


C:\Users\reesh\anaconda3\envs\hands\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


242/242 [==============================] - 2s 6ms/step - loss: 1.0407 - val_loss: 0.7215
Epoch 2/100
242/242 [==============================] - 1s 5ms/step - loss: 0.5469 - val_loss: 0.5096
Epoch 3/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4861 - val_loss: 0.5524
Epoch 4/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4663 - val_loss: 0.5304
Epoch 5/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4358 - val_loss: 0.4307
Epoch 6/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4120 - val_loss: 0.4262
Epoch 7/100
242/242 [==============================] - 1s 5ms/step - loss: 0.4033 - val_loss: 0.4128
Epoch 8/100
242/242 [==============================] - 1s 5ms/step - loss: 0.3874 - val_loss: 0.4059
Epoch 9/100
242/242 [==============================] - 1s 5ms/step - loss: 0.3795 - val_loss: 0.3914
Epoch 10/100
242/242 [==============================] - 1s 5ms/step - loss: 0.3737 - val_loss: 0.3901
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x0000020977927160>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000020A4E6750C0>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])})

In [17]:
rnd_search_cv.best_params_

{'learning_rate': 0.004514104244558324, 'n_hidden': 3, 'n_neurons': 79}

In [18]:
rnd_search_cv.best_score_

-0.311768760283788

In [19]:
model = rnd_search_cv.best_estimator_.model